# High-throughput Screening for Novel Photoferroic Perovskites


In this year's final project for the course we address the problem of discovering novel perovskites materials that combine ferroelectricity with good light harvesting properties (photoferroics). A solar cell based on a photoferroic material can achieve efficiencies well beyond the limit determined by the Schockley-Queisser model because of a different mechanism for the generation of the photovoltage. More information on this as well as the criteria required by this technology can be found in a recent work (https://iopscience.iop.org/article/10.1088/2515-7655/ab428c).

In this project, we establish a screening procedure to identify perovskite materials with the formula A2A’B2X7, where A = Li, Na, and K; A’ = Mg, Ca, Sr, and Ba; B = Ta and Nb; and X = O, S, and Se. This procedure can be split in different steps, for each material:

1) relax 5 different prototypes (available from the `prototypes` folder) with PBEsol;

2) compare the total energies of the relaxations in (1) and if the gap of the ground state (GS) structure is larger than zero, perform a more strict relaxation for the GS;

3) calculate band gap and absorption spectrum of the GS, with a more accurate exchange-correlation functional (GLLB-SC);

4) calculate the polarization for the GS;

5) select materials based on the descriptors: convex hull < 0.1 eV/atom; 1 < band gap < 2.5 eV; polarization.

Remember that to submit the calculations to the gbar, you have to run the cell with the `%%writefile script.py` command as the first line in the cell to write `script.py` as a file on the cluster (you can also write the script directly on the cluster by using an editor in the terminal). Then you submit the calculation using the `!qsub.py -t T -p NPROC script.py` command in a cell (or in the terminal directly if you remove the exclamation mark `!`. For reference see: ase-gpaw.ipynb#Submitting-calculations 

### Note: please keep the name of the files as indicated. It will make our life much easier when we will write a paper about this screening project.

### 1) Relaxation
You can find 5 prototypes in the folder `prototypes` (space groups Pnma, CmCm, Cmc21, P42mnm, and I4mmm; some are centrosymmetric and some are not).

You need to decorate, i.e. replace the chemical elements, to investigate the structures given to you. For example, the cif files now describes a structure with formula Li2SrNb2O7, but you have to calculate K2CaTa2O7, you have to replace Li→K, Sr→Ca, …
To cover the full space of investigation, we divide the materials to calculate as follow:
	    
    Smobin Vincent: A' = Ca, X = O, all A (Li, Na, K), all B (Ta, Nb)
    Asnake Sahle: A' = Sr, X = O, all A, all B
    Fangyuan Diao: A' = Ba, X = O, all A, all B
    Seokhyun Choung: A' = Mg, X = S, all A, all B
    Ali Hassan: A' = Ca, X = S, all A, all B
    Laura Friedeheim: A' = Sr, X = S, all A, all B
    Wei Huang: A' = Ba, X = S, all A, all B
    Nicola Mazzanti: A' = Mg, X = Se, all A, all B
    Meng Shi: A' = Ca, X = Se, all A, all B
    Jiahuan Tong: A' = Sr, X = Se, all A, all B
    Stefan Pollock: A' = Ca, X = Se, all A, all B
    if someone is missing from the list above, please run: A' = Mg, X = O, all A, all B

You run a relaxation for each structure with the following script:

In [ ]:
import os,sys,re

from ase.io import read, write
from ase.constraints import UnitCellFilter
from ase.optimize.bfgslinesearch import BFGSLineSearch
from ase.visualize import view
from gpaw import GPAW, PW, FermiDirac
from gpaw import Mixer

# read the prototype and decorate it with the desired elements 
atoms = read('xxx') # where xxx is the structure/prototype
formula = ""
prototype = ""

# The unit cell size will change after substitution of elements. A
# crude estimate of the new cell size could be the old cell times the
# ratio between new and old atomic radii. Something like the following
# could work:
# new_cell = atoms.cell * (final_radii / init_radii) ** (1/3)
# atoms.set_cell(new_cell, scale_atoms=True)
# covalent radii can be found in ase.data i.e.
# from ase.data import covalent_radii

# Substiting elements to get the correct formula with something like:
# atoms.symbols[[a.index for a in atoms if a.symbol == 'Li']] = 'x'


# now perform the relaxation
name = 'relax_'+formula+'-'+prototype

# setup the calculator
calc_gpaw = GPAW(mode=PW(550),
                 xc='PBEsol',
                 txt='{0}.txt'.format(name),
#                     eigensolver='cg',
                 mixer=Mixer(beta=0.05, nmaxold=5, weight=50.0),
                 kpts={'density': 2.5, 'gamma': False},
                 occupations=FermiDirac(width=0.05))

atoms.set_calculator(calc_gpaw)
# relaxation using a mask
sf = UnitCellFilter(atoms,mask=[1,1,1,0,0,0])
opt = BFGSLineSearch(sf,trajectory ='{0}.traj'.format(name), logfile = '{0}.log'.format(name), restart = '{0}.pckl'.format(name))
opt.run(fmax=0.1) # until forces < 0.1 eV/atom
calc_gpaw.write('{0}.gpw'.format(name))



### 2) Find ground state, band gap, and more strict relaxation

Once that all the prototypes for a given composition are relaxed, you need to find **the most stable prototype for that composition, we term this** the ground state (GS). You need then to calculate the band gap for the GS, which can be done with:

In [ ]:
from gpaw import *
from ase.dft import bandgap

atoms, calc = restart('relax_'+formula+'-'+prototype+'.gpw',txt=None)
gap, p1, p2 = bandgap.bandgap(calc)

If the GS has a band gap > 0, then you perform a more strict relaxation with a script similar to the one for the relaxation above, but with these modifications:

In [ ]:
name = 'strict-relax_'+formula+'-'+prototype
opt.run(fmax=0.05)

### 3) Band gap and absorption spectrum with GLLBSC

With a thoroughly relaxed GS we need to determine the band gap accurately with the GLLB-SC that are known to yield accurate band gaps. You can find more details about the functional used here: https://journals.aps.org/prb/abstract/10.1103/PhysRevB.82.115106 and https://pubs.rsc.org/en/content/articlelanding/2012/ee/c1ee02717d#!divAbstract


In [ ]:
name = 'gllbsc_'+formula+'-'+prototype

# no magnetic implemented
if atoms.get_initial_magnetic_moments().any():
    exit()

calc_gllbsc = GPAW(mode=PW(550),
                   xc='GLLBSC',
                   txt= '{0}.txt'.format(name),
                   nbands=int(ne/2+30),
                   convergence={'bands':-20},
                   maxiter = 2500,
                   kpts={'density': 4, 'gamma': True},
                   mixer=Mixer(beta=0.05, nmaxold=5, weight=50.0),
                   occupations=FermiDirac(width=0.05))
atoms.set_calculator(calc_gllbsc)
atoms.get_potential_energy()
response = atoms.calc.hamiltonian.xc.xcs['RESPONSE']
response.calculate_delta_xc()
Eks, deltaxc = response.calculate_delta_xc_perturbation()
atoms.calc.write(name+'.gpw','all')
o = open(name+'_dxc.txt','w')
o.write(str(deltaxc))
o.close()

where `ne` is the number of electrons that you can extract from the PBEsol calculator with:

In [ ]:
calc_gpaw = GPAW(name+'.gpw', txt=None)
atoms = calc_gpaw.get_atoms()
ne = calc_gpaw.get_number_of_electrons()

Very importantly, the derivative discontinuity is saved in a txt file. Why is the derivative discontinuity important for the calculation of the electronic properties?

Once that you have calculated the band gap and the derivative discontinuity, you can calculate the full absorption spectrum. This can be done with the script:

In [ ]:
name = 'gllbsc_'+formula+'-'+prototype

ecut = 50
df = DielectricFunction(calc=name+'.gpw',
                        frequencies=np.linspace(0, 5, 501),
                        hilbert=False,
                        nblocks=ncores,
                        eta=0.05,
                        eshift=deltaxc,
                        ecut=ecut,
                        txt='gllbsc_'+formula+'-'+prototype+'_abs-spectrum.txt'
                        )
df.get_dielectric_function(direction='x', filename=name+'df_x.csv')
df.get_dielectric_function(direction='y', filename=name+'df_y.csv')
df.get_dielectric_function(direction='z', filename=name+'df_z.csv')

where ncores is the number of cpus per node and deltaxc is the derivative discontinuity that you can read from the file generated in point 3 (name+'_dxc.txt').

The adsorption spectum for the three directions (x,y,z) is saved in the csv files. As described here https://wiki.fysik.dtu.dk/gpaw/tutorials/dielectric_response/dielectric_response.html, you need to plot the first and third column (Im(epsilon) vs Omega).

Is the band gap a good desscriptor for the efficiency of this material? 

You can read more about the absorption spectrum calcultions for light harvesting materials here https://pubs.rsc.org/en/content/articlelanding/2014/TA/C5TA01586C#!divAbstract.

### 4) Polarization  
##### (you can run this at the same time as point 3, as the input structure is the strict_relax geometry)

At the same time as we calculate the electronic properties, we need to address the question whether the material is ferroelectric or not. This can be done with the following calculator:

In [ ]:
from gpaw.berryphase import get_polarization_phase
from ase.units import _e

name = 'strict_relax_'+formula+'-'+prototype
calc_gpaw = GPAW(name+'.gpw', txt=None)
atoms = calc_gpaw.get_atoms()

phi_c = get_polarization_phase(name+'.gpw')
cell_v = np.diag(atoms.get_cell()) * 1.0e-10
V = np.prod(cell_v)
P_v = (phi_c / (2 * np.pi) % 1) * cell_v * _e / V
P0_v = cell_v * _e / V

o = open(name+'_polarization.txt','w')
o.write(str(P_v)+' '+str(P0_v))
o.close()

The txt file includes 2 arrays, the first gives the polarization and the second represent the n-quantum (explanation follows). The polarization within the Berry phase approximation used here, is defined modulus a quantum. This means that the resulting polarization (we are interested in the z-direction) is defined as $P_{net} = P-n*quantum$, where $n$ is an largest integer number that will give you a positive net polarization $P_{net}$. As an example, if the polarization $P = 2.5$ and the $quantum = 1$, the net polarization $P_{net}$ is 0.5, i.e. $P_{net}=2.5-n*1$, because n will be 2. Is the GS material ferroelectric?

### Convex hull stability

Is the candidate material stable? To address this point, you have to calculate the phase diagram (convex hull) following this example: https://wiki.fysik.dtu.dk/ase/ase/phasediagram/phasediagram.html

The pool of reference systems has already been calculated and available at `references_pbesol.db`

In [ ]:
## Cell to calculate stability vs convex hull

## Get the references like this
from ase.db import connect
db = connect('references_pbesol.db')
refs = [(row.formula, row.energy) for row in db.select('Li>0')]

### Descriptors for the electronic properties

Does the candidate material have the potential to be used in a solar cell, i.e. band gap between 1. and 2.5 eV? Can we use this material in a thin film device (high light absorption properties at the band gap) or not? Discuss this comparing the band gap with the shape of the adsorption spectrum.

### Trends and correlations

Taking inspiration from:
https://iopscience.iop.org/article/10.1088/0965-0393/22/5/055007
https://www.sciencedirect.com/science/article/abs/pii/S0143720806003287?via%3Dihub

identify and discuss possible trends in the set of materials that you have investigated.

### Presentation

For the exam, you have to prepare a presentation describing your results.